In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [2]:
# load YAML 
with open('data.yaml', mode= 'r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)
    
labels = data_yaml['names']
print(labels)

#Load the image



['person', 'car', 'chair', 'bottle', 'pottedplant', 'bird', 'dog', 'sofa', 'bicycle', 'horse', 'boat', 'motorbike', 'cat', 'tvmonitor', 'cow', 'sheep', 'aeroplane', 'train', 'diningtable', 'bus']


In [3]:
# Load YOLO model
yolo = cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)

In [4]:
#Load the image
img = cv2.imread('./street_image.jpg')
image = img.copy()

row, col , d = image.shape

# get the YOLO predictions from the image
#step-1: convert image into square image(array)
max_rc =max(row,col)
input_image = np.zeros((max_rc,max_rc,3), dtype=np.uint8)
input_image[0:row,0:col]= image
#step-2: get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image ,1/255,(INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True,crop=False)
yolo.setInput(blob)
pred = yolo.forward() #detection or predition from YOLO



In [5]:
print(pred.shape)

(1, 25200, 25)


In [9]:
# non maximum supression
#sterp-1: filter detection based on confidence (0.4) and probability score 0.25
detections = pred[0]
boxes = []
confidences = []
classes = []

#width and height of the images {input image}
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] #confidence of deetectioon an object
    if confidence > 0.4 :
        class_score = row[5:].max() #maximum probability from 20 objects
        class_id = row[5:].argmax() #get the index position at which max probabilitu occur
        
        if class_score > 0.25:
            cx,cy,w,h = row[0:4]